In [2]:
#pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357248 sha256=2fda0b18744a37fe6398beac8322bce1cf2cd7d2310c558c5615c6e38c15e863
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [14]:
import pandas as pd
from surprise import Dataset, Reader, SVDpp, accuracy
from surprise.model_selection import train_test_split, GridSearchCV

In [15]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
#movies = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/movies.csv', delimiter= '::', encoding= 'latin1')
#users = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/users.csv', delimiter='::')

In [16]:
test

,TestDataID,UserID,MovieID
0,1,796,594
1,2,796,1270
2,3,796,1907
3,4,3203,2126
4,5,3203,292
...,...,...,...
100331,100332,2683,111
100332,100333,2683,3334
100333,100334,2683,2717
100334,100335,2683,1784


In [17]:
reader = Reader(rating_scale= (1,5))
data = Dataset.load_from_df(train[['UserID', 'MovieID', 'Rating']], reader)

In [18]:
train_set, test_set = train_test_split(data, test_size=0.3)

In [19]:
svd = SVDpp()
svd.fit(train_set)

In [20]:
test_pred = svd.test(test_set)
accuracy.rmse(test_pred)

RMSE: 0.8741


0.8740865162145134

In [21]:
predictions = []

for _, row in test.iterrows():
    user = row['UserID']
    movie = row['MovieID']

    res = svd.predict(user, movie)

    predictions.append((row['TestDataID'], res.est))

submission = pd.DataFrame(predictions, columns=['TestDataID', 'Rating'])

submission['Rating'] = submission['Rating'].round()


In [23]:
submission.to_csv('/content/sub1_svdpp.csv', index_label= 'id', index = False)